### load AF model evaluation metrics ###

In [1]:
import os 
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
def parse_AF_eval(plddt_file, confidence_file):
    if not os.path.exists(plddt_file) or not os.path.exists(confidence_file):
        print(f"File not found: {plddt_file} or {confidence_file}")
        return
    with open(plddt_file, 'r') as f:
        pdb_plddt_dict = json.load(f)
    with open(confidence_file, 'r') as f:
        pdb_confidence_dict = json.load(f)
    
    eval_dict = {
        'mean_plddt': np.mean(pdb_plddt_dict['atom_plddts']),
        'All_mean_pae': np.mean(pdb_plddt_dict['pae']),
        'iptm': pdb_confidence_dict['iptm'],
        'ptm': pdb_confidence_dict['ptm']
    }
    return eval_dict

In [12]:
AF_struct_dir = '/home/lwang/models/HDX_LSTM/data/COVID_SPIKE/AF_VH16_VL106'
seeds = range(1, 11)
protien_name = 'vh16_vl106'
model_evals = {}

for i in seeds:
    for j in range(0, 5):
        pdb_plddt = f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_full_data_{j}.json'
        pdb_confidence= f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_summary_confidences_{j}.json'
        model_evals[(i,j)]= parse_AF_eval(pdb_plddt, pdb_confidence)
print(model_evals)

{(1, 0): {'mean_plddt': 76.62606447280054, 'All_mean_pae': 10.887116370808679, 'iptm': 0.32, 'ptm': 0.57}, (1, 1): {'mean_plddt': 77.47076561450638, 'All_mean_pae': 10.574756081525313, 'iptm': 0.32, 'ptm': 0.58}, (1, 2): {'mean_plddt': 76.35648757555406, 'All_mean_pae': 11.058624589086126, 'iptm': 0.31, 'ptm': 0.56}, (1, 3): {'mean_plddt': 76.36276695768973, 'All_mean_pae': 11.107392504930967, 'iptm': 0.3, 'ptm': 0.56}, (1, 4): {'mean_plddt': 77.06720617864339, 'All_mean_pae': 10.905051939513479, 'iptm': 0.29, 'ptm': 0.57}, (2, 0): {'mean_plddt': 81.84419073203492, 'All_mean_pae': 7.005677843523998, 'iptm': 0.73, 'ptm': 0.8}, (2, 1): {'mean_plddt': 80.87838817998657, 'All_mean_pae': 7.653659434582512, 'iptm': 0.69, 'ptm': 0.77}, (2, 2): {'mean_plddt': 80.37238415043653, 'All_mean_pae': 7.846301117685733, 'iptm': 0.69, 'ptm': 0.77}, (2, 3): {'mean_plddt': 80.62885157824043, 'All_mean_pae': 7.611926364234057, 'iptm': 0.69, 'ptm': 0.78}, (2, 4): {'mean_plddt': 79.43284754869039, 'All_mean

In [32]:
model_evals_df = pd.DataFrame(model_evals).T
model_evals_df = model_evals_df.reset_index()
display(model_evals_df.sort_values('mean_plddt', ascending=False)[:10])

top_5_rows = model_evals_df.sort_values('mean_plddt', ascending=False).head(5)
ij_pair = list(top_5_rows[['level_0', 'level_1']].itertuples(index=False, name=None))
select_models = [f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{j}.cif' for i,j in ij_pair]

,level_0,level_1,mean_plddt,All_mean_pae,iptm,ptm
5,2,0,81.844191,7.005678,0.73,0.80
16,4,1,81.543150,6.921602,0.74,0.81
15,4,0,81.518019,7.083550,0.75,0.81
17,4,2,81.339872,7.544650,0.71,0.79
22,5,2,80.906951,7.806656,0.69,0.78
6,2,1,80.878388,7.653659,0.69,0.77
20,5,0,80.670175,7.914932,0.69,0.78
8,2,3,80.628852,7.611926,0.69,0.78
19,4,4,80.491974,7.444757,0.69,0.78
7,2,2,80.372384,7.846301,0.69,0.77


In [35]:
import shutil
from Bio.PDB import MMCIFParser, PDBIO

output_dir = '/home/lwang/models/HDX_LSTM/data/COVID_SPIKE/structure'
for model in select_models:
    parser = MMCIFParser(QUIET=True)
    structure = parser.get_structure('model', model)
    pdb_filename = os.path.join(output_dir, os.path.basename(model).replace('.cif', '.pdb'))
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_filename)

### cluster AF models based on RMSD ###

In [20]:
from Bio.PDB import PDBParser, MMCIFParser
import numpy as np

def extract_chain_coordinates(structure_file, chain_id, atom_name=["CA"]):
    """
    Extracts the coordinates of specified atoms in specified chains from a PDB or mmCIF file.
    
    :param structure_file: Path to the PDB or mmCIF file
    :param chain_id: List of chain IDs to extract (or a single chain ID as a string)
    :param atom_name: List of atom names to filter (e.g., ["CA", "CB"]). Defaults to ["CA"].
    :return: A NumPy array of shape (n_atoms, 3) containing the atomic coordinates
    """
    # Determine parser based on file extension
    if structure_file.endswith('.pdb'):
        parser = PDBParser(QUIET=True)
    elif structure_file.endswith('.cif') or structure_file.endswith('.mmcif'):
        parser = MMCIFParser(QUIET=True)
    else:
        raise ValueError("Unsupported file format. Please provide a .pdb or .cif/.mmcif file.")
    
    structure = parser.get_structure('structure', structure_file)
    
    # Ensure chain_id and atom_name are lists
    if isinstance(chain_id, str):
        chain_id = [chain_id]
    if isinstance(atom_name, str):
        atom_name = [atom_name]

    # Extract coordinates with chain and atom filters
    coordinates = []
    for id in chain_id:
        chain = structure[0][id]
        for atom in chain.get_atoms():
            if atom.get_name() in atom_name:
                coordinates.append(atom.coord)
    
    return np.array(coordinates)

def calculate_RMSD(coord1, coord2):
    """
    Calculates the root-mean-square deviation (RMSD) between two sets of atomic coordinates.
    
    :param coord1: NumPy array of shape (n_atoms, 3) containing the first set of atomic coordinates
    :param coord2: NumPy array of shape (n_atoms, 3) containing the second set of atomic coordinates
    :return: The RMSD between the two coordinate sets
    """
    # Check if the number of atoms is the same in both coordinate sets
    if coord1.shape[0] != coord2.shape[0]:
        raise ValueError("The number of atoms in the two coordinate sets is different.")
    
    # Calculate the RMSD
    diff = coord1 - coord2
    rmsd = np.sqrt(np.mean(np.sum(diff**2, axis=1)))
    
    return rmsd

In [24]:
RMSD_dict = {}
chain_ids = ['A', 'B']
for i in seeds:
    for j in range(0, 4):
        ref_coord = extract_chain_coordinates(f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{j}.cif', chain_ids)
        for k in range(j+1, 5):
            compare_coord = extract_chain_coordinates(f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{k}.cif', chain_ids)
            RMSD_dict[(i,j,k)] = calculate_RMSD(ref_coord, compare_coord)
print(RMSD_dict)


{(1, 0, 1): 27.454308, (1, 0, 2): 23.082153, (1, 0, 3): 20.98222, (1, 0, 4): 22.817755, (1, 1, 2): 27.783411, (1, 1, 3): 28.234781, (1, 1, 4): 28.79486, (1, 2, 3): 10.040766, (1, 2, 4): 14.338803, (1, 3, 4): 16.893402, (2, 0, 1): 28.479342, (2, 0, 2): 27.088152, (2, 0, 3): 21.378466, (2, 0, 4): 24.406746, (2, 1, 2): 12.121987, (2, 1, 3): 19.387264, (2, 1, 4): 23.792034, (2, 2, 3): 14.864815, (2, 2, 4): 20.279856, (2, 3, 4): 27.091763, (3, 0, 1): 15.288367, (3, 0, 2): 28.163675, (3, 0, 3): 24.094091, (3, 0, 4): 23.007807, (3, 1, 2): 23.993904, (3, 1, 3): 28.047935, (3, 1, 4): 22.999348, (3, 2, 3): 23.56113, (3, 2, 4): 26.504272, (3, 3, 4): 27.112698, (4, 0, 1): 24.134193, (4, 0, 2): 26.506433, (4, 0, 3): 24.022974, (4, 0, 4): 26.956429, (4, 1, 2): 23.725182, (4, 1, 3): 25.331045, (4, 1, 4): 21.758139, (4, 2, 3): 27.843176, (4, 2, 4): 20.373838, (4, 3, 4): 27.216896, (5, 0, 1): 21.154747, (5, 0, 2): 25.772144, (5, 0, 3): 23.021336, (5, 0, 4): 24.64439, (5, 1, 2): 19.861696, (5, 1, 3): 6.

In [28]:
from DockQ.DockQ import load_PDB, run_on_all_native_interfaces
mapping = {"A": "A", "B": "B"}
pdb_files = [f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{j}.cif' for i in seeds for j in range(0, 5)]
print(len(pdb_files))

for i in range(0, len(pdb_files)):
    for j in range(i+1, len(pdb_files)):
        pdb1 = load_PDB(pdb_files[i])
        pdb2 = load_PDB(pdb_files[j])
        result = run_on_all_native_interfaces(pdb1, pdb2, chain_map=mapping)
        print(result)
        break
'''for i in seeds:
    for j in range(0, 4):
        native = load_PDB(f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{j}.cif')
        for k in range(j+1, 5):
            model = load_PDB(f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{k}.cif')
            result = run_on_all_native_interfaces(model, native, chain_map=mapping)
            RMSD_dict[(i,j,k)] = result[0].get('Lrms', np.nan)
            break'''

50


"for i in seeds:\n    for j in range(0, 4):\n        native = load_PDB(f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{j}.cif')\n        for k in range(j+1, 5):\n            model = load_PDB(f'{AF_struct_dir}/{protien_name}_seed{i}/fold_{protien_name}_seed{i}_model_{k}.cif')\n            result = run_on_all_native_interfaces(model, native, chain_map=mapping)\n            RMSD_dict[(i,j,k)] = result[0].get('Lrms', np.nan)\n            break"

### HDOCK: rigid body global docking ###

In [40]:
import subprocess

def run_HDOCK(hdock_path, receptor, ligand, out_file):
    command = f"{hdock_path} {receptor} {ligand} -out {out_file}"
    subprocess.run(command, shell=True)

In [ ]:
import os
os.chdir('/home/lwang/models/HDOCKlite-v1.1')
protien_name = 'vh16_vl106'
pdb_dir = f'./pdb'
os.mkdir('./output') if not os.path.exists('./output') else None

receptors = ['Wuhan_spike_clean.pdb', 'Omicron_spike_clean.pdb', 'Delta_spike_clean.pdb']
ligands = [file for file in os.listdir(f"./pdb") if file.startswith(f"fold_{protien_name}")]

for receptor in receptors:
    for ligand in ligands:
        out_file = f'./output/{receptor.split("_")[0]}_{ligand.split(".")[0]}.out'
        run_HDOCK('./hdock', f'{pdb_dir}/{receptor}', f'{pdb_dir}/{ligand}', out_file)
